# Xopt basic example

Xopt optimization problems can be defined via one of two methods:
- a yaml text file (for limiting the amount of python script writing and/or setting up simulation runs)
- a simple python script (for those who prefer to use python directly)

Here we will demonstrate how both of these techniques can be used to solve a relatively simple  constrained optimization problem.

$n=2$ variables:
$x_i \in [0, \pi], i=1,2$

Objective:
- $f(x) = \sum_i x_i$

Constraint:
- $g(x) = -x_1^2 -x_2^2 + 1 \le 0$

## Xopt Components
The definition of the Xopt object requires 3 parts, listed below:
- The `Evaluator` object, which evaluates input points using the arbitrary function
specified by the `function` property.
- The `Generator` object, which, when given data that has been evaluated, generates
future points to evaluate using the evaluator.
- The `VOCS` (variables, objectives, constraints, statics) object, which specifies the
input domain, the objectives, constraints and constants passed to the evaluator
function.


# Import 

In [ ]:
from xopt import Evaluator
from xopt import VOCS
from xopt import Xopt
from xopt.generators import list_available_generators
from xopt.generators import get_generator
import math

## Defining Xopt components using python
We first examine how one would create and configure and Xopt optimization run using
python. This can also be done via a YAML file (see the next section).

### Define the objective function and the evaluator
Note that the objective function takes in a dict of variable values and returns a dict of objective return values. The keys of the input and output dictionaries must contain the keys we will specify in VOCS (see below).

In [ ]:
def evaluate_function(inputs: dict) -> dict:
    objective_value = inputs["x1"] ** 2 + inputs["x2"] ** 2
    constraint_value = -(inputs["x1"] ** 2) - inputs["x2"] ** 2 + 1
    return {"f": objective_value, "g": constraint_value}


evaluator = Evaluator(function=evaluate_function)

### Define VOCS
Here we define the names and ranges of input parameters, the names and settings of
objectives, and the names and settings of constraints. Note that the keys here should
 be referenced in the evaluate function above.

In [ ]:
vocs = VOCS(
    variables={"x1": [0, math.pi], "x2": [0, math.pi]},
    objectives={"f": "MINIMIZE"},
    constraints={"g": ["LESS_THAN", 0]},
)

### Define the Generator
First lets see which generators are available for use.

In [ ]:
list_available_generators()

Here we will use the simplest generator that is defined by Xopt, random number generation.

In [ ]:
# get the docstring for the random generator
print(get_generator("random").__doc__)

# use the get generator method to get the random number generator
generator = get_generator("random")(vocs=vocs)

###  Combine into Xopt object

In [ ]:
X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator)

## Defining Xopt object from yaml file
Alternatively, it might be more useful to define the Xopt object from a text file or
YAML string. We replicate the code above with the YAML file below.

In [ ]:
# Make a proper input file.
YAML = """
evaluator:
    function: __main__.evaluate_function

generator:
    name: random

vocs:
    variables:
        x1: [0, 3.14159]
        x2: [0, 3.14159]
    objectives: {f: MINIMIZE}
    constraints:
        g: [LESS_THAN, 0]

"""

In [ ]:
# create Xopt object.
X_from_yaml = Xopt.from_yaml(YAML)

## Introspection
Objects in Xopt can be printed to a string or dumped to a text file for easy
introspection of attributes and current configuration.

In [ ]:
# Convenient representation of the state.
X

## Evaluating randomly generated or fixed inputs.
The main Xopt object has a variety of means for evaluating random or fixed points.
This is often used to initialize optimization, but can be used independently of any
generator. Results from evaluations are stored in the `data` attribute. Data can also
 be explictly added to the Xopt object (and by extension the generator attached to
 the xopt object by calling `X.add_data()`.

In [ ]:
# randomly evaluate some points and add data to Xopt object
X.random_evaluate(5)

In [ ]:
# evaluate some points additionally
points = {"x1": [1.0, 0.5, 2.25], "x2": [0, 1.75, 0.6]}
X.evaluate_data(points)

In [ ]:
# examine the data stored in Xopt
X.data

## Optimization
Xopt conducts a single iteration of optimization by calling `X.step()`. Inside this
function Xopt will generate a point (or set of points) using the generator object,
then send the point to be evaluated by the evaluator. Results will be stored in the
data attribute.

In [ ]:
# Take one step (generate a single point)
X.step()

In [ ]:
# examine the results
X.data

In [ ]:
# take a couple of steps and examine the results
for _ in range(10):
    X.step()
X.data

## Find and evaluate the best point from `X.data`

In [ ]:
idx, val, params = X.vocs.select_best(X.data)
print(f"best objective value {val}")
print(f"best point {params}")

X.evaluate_data(params)

## Visualization
Finally, we can visualize the objectives and variables to monitor optimization or
visualize the results

In [ ]:
# view objective values
X.data.plot(y=X.vocs.objective_names)

# view variables values
X.data.plot(*X.vocs.variable_names, kind="scatter")

# you can also normalize the variables
X.vocs.normalize_inputs(X.data).plot(*X.vocs.variable_names, kind="scatter")